## Desenvolvimento do modelo de predição de preço

Exploração inicial de modelos de base

In [1]:
import pandas as pd
from dagshub.data_engine import datasources
import mlflow
import dagshub
from sklearn.model_selection import train_test_split
import mlflow.sklearn
import mlflow.catboost
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import mlflow.models.signature
from mlflow.models import infer_signature
from catboost import CatBoostRegressor

In [4]:
ds = datasources.get_datasource("pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl","processed")

In [5]:
ds.all().dataframe

Output()

,path,datapoint_id,dagshub_download_url,media type,size
0,laptop-price-brl-processed.csv,102408450,https://dagshub.com/api/v1/repos/pedromonnt/fi...,text/plain,52559


In [6]:
res = ds.head()

for dp in res:
    dataset_url = dp.download_url

Output()

In [7]:
dataset_url

'https://dagshub.com/api/v1/repos/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl/raw/main/data/processed/laptop-price-brl-processed.csv'

In [8]:
df = pd.read_csv(dataset_url)
df.head()

,ram_gb,ssd,hdd,graphic_card_gb,warranty,touchscreen,price,brand_asus,brand_dell,brand_hp,...,processor_name_ryzen 7,ram_type_ddr4,ram_type_other,os_other,os_windows,os_bit_32-bit,os_bit_64-bit,weight_casual,weight_gaming,weight_thinnlight
0,4,0,1024,0,0,0,2321,1,0,0,...,0,1,0,0,1,0,1,1,0,0
1,4,0,1024,0,0,0,2613,0,0,0,...,0,1,0,0,1,0,1,1,0,0
2,4,0,1024,0,0,0,2680,0,0,0,...,0,1,0,0,1,0,1,1,0,0
3,8,512,0,2,0,0,4689,1,0,0,...,0,1,0,0,1,1,0,1,0,0
4,4,0,512,0,0,0,1808,1,0,0,...,0,1,0,0,1,0,1,1,0,0


## Desenvolvimento e experimentos de modelos

In [9]:
dagshub.init(repo_owner="pedromonnt", repo_name="fiap-ds-mlops-10dtsr-laptop-pricing-brl", mlflow = True)

Initialized MLflow to track repo "pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl"

Repository pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl initialized!

In [10]:
mlflow.autolog()

2025/07/09 18:38:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/07/09 18:38:55 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/09 18:38:55 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [12]:
features = list(df.columns)
features.remove("price")

In [13]:
features

['ram_gb',
 'ssd',
 'hdd',
 'graphic_card_gb',
 'warranty',
 'touchscreen',
 'brand_asus',
 'brand_dell',
 'brand_hp',
 'brand_lenovo',
 'brand_other',
 'processor_brand_amd',
 'processor_brand_intel',
 'processor_brand_m1',
 'processor_name_core i3',
 'processor_name_core i5',
 'processor_name_core i7',
 'processor_name_other',
 'processor_name_ryzen 5',
 'processor_name_ryzen 7',
 'ram_type_ddr4',
 'ram_type_other',
 'os_other',
 'os_windows',
 'os_bit_32-bit',
 'os_bit_64-bit',
 'weight_casual',
 'weight_gaming',
 'weight_thinnlight']

In [14]:
X = df[features]

In [15]:
len(features)

29